In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.maxbasis import MaxBasis
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay, easeFunction

objc[1842]: Class GLFWApplicationDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x294966e28) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2af396ea0). One of the two will be used. Which one is undefined.
objc[1842]: Class GLFWWindowDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x294966db0) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2af396ec8). One of the two will be used. Which one is undefined.
objc[1842]: Class GLFWContentView is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x294966e50) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-pac

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths1 = np.arange(390, 701, 1)
wavelengths5 = np.arange(400, 701, 5)
wavelengths10 = np.arange(400, 701, 10)

In [5]:
wavelengths = np.arange(400, 710, 10)
d65 = Illuminant.get("D65")

In [6]:
trichromat = Observer.trichromat(wavelengths=wavelengths5, illuminant=d65) 
maxbasis_tri = MaxBasis(trichromat, verbose=True)
# viz_tri = PSWrapper(trichromat, maxbasis_tri, itemsToDisplay=PSWrapper.ItemsToDisplay.MESH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True)

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True) 
maxbasis = MaxBasis(tetrachromat, verbose=True)
viz = PSWrapper(tetrachromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True) 

100%|██████████| 58/58 [00:00<00:00, 653.72it/s]
/Users/jessicalee/Projects/generalized-colorimetry/code/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")
100%|██████████| 28/28 [00:00<00:00, 48.13it/s]


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1


100%|██████████| 8990/8990 [00:19<00:00, 465.68it/s]


In [7]:
refs, discrete_points, rgbs, lines = maxbasis.getDiscreteRepresentation()
# fig, axs = plt.subplots(4, 4, figsize=(8, 8))

# for i in range(4):
#     for j in range(4):
#         axs[i, j].plot(wavelengths, refs[i*4+j].data, c=rgbs[i*4 + j])
#         axs[i, j].fill_between(wavelengths, refs[i*4+j].data, color=rgbs[i*4 + j])
#         axs[i, j].set_xlabel('Wavelength')
#         axs[i, j].set_ylabel('Refs')
#         axs[i, j].set_title(f'Ref {i*4+j}')

# plt.tight_layout()
# plt.show()


In [8]:
viz.ps.set_ground_plane_mode("none")
dirname = "./outputs/perception"

def doNothing(j):
    pass

viz.renderObjectsPS(mesh_alpha=0.8, lattice_alpha=1)
viz.renderQArrow()
mat = viz._getTransformQUpDir()
viz.ps.get_surface_mesh("mesh").set_enabled(True)
viz.ps.get_surface_mesh("qarrow").set_enabled(False)
viz.ps.get_surface_mesh("lattice").set_enabled(False)

viz.ps.get_surface_mesh("mesh").set_transform(mat)
viz.ps.get_surface_mesh("lattice").set_transform(mat)
viz.ps.get_surface_mesh("qarrow").set_transform(mat)

ref_points = (mat[:3, :3]@((viz.HMatrix@discrete_points.T)[1:, :])).T
# viz.ps.show()

r = 4
theta = 65
phi = 0
frames_per_path = 30
rots_per_sec = 0.33
opacity_baseline = 0.2

# 1. hue sphere is differnet from the hue circle
viz.renderRotationAroundZ(doNothing, dirname,  rots_per_sec, r, theta)
viz.ps.get_surface_mesh("lattice").set_enabled(True)
viz.ps.get_surface_mesh("lattice").set_transparency(opacity_baseline)

# 2. Travel to Lattice Points with 4 neighbors
magenta = ref_points[7]
orange = ref_points[10]

r_theta_phis = [orange, magenta]
polar = np.array([PSWrapper.cartesianToPolar(v[0], v[1], v[2]) for v in r_theta_phis])
polar[:, 0] = r

# Debug
# cartesian = np.array([PSWrapper.polarToCartesian(v[0], v[1], v[2]) for v in polar])
# viz.ps.look_at(cartesian[1], [0, 0, 0])
# viz.ps.show()

last_phi = polar[-1, -1]
path = [[r, theta, phi]] + polar.tolist() + [[r, 90, last_phi], [r, 5, last_phi], [r, 90, last_phi], [r, 180-5, last_phi], [r, 90, last_phi]]
path_opacity = np.cos(2 * np.linspace(0, 3 * np.pi, 3 * frames_per_path))/2.5 + 0.6
path_opacity[:15] = opacity_baseline
path_opacity[-15:] = opacity_baseline
path_opacity = np.concatenate([path_opacity, opacity_baseline * np.ones(4 * frames_per_path)])

viz.renderPath(dirname, frames_per_path, path, path_opacity, video_save_offset=int(1/rots_per_sec))

arrow set to true


In [9]:
exportAndPlay(dirname)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex 